In [17]:
#Modified code from Web Data: Amazon reviews
import gzip
import pandas as pd

def parse(filename):
  f = gzip.open(filename, 'r')
  entry = {}
  for l in f:
    l = l.decode('utf8').strip()
    colonPos = l.find(':')
    if colonPos == -1:
      yield entry
      entry = {}
      continue
    eName = l[:colonPos]
    rest = l[colonPos+2:]
    entry[eName] = rest
  yield entry

#create list with dictionaries, from which we will create dataframe
data = []
for e in parse("data\Cell_Phones_&_Accessories.txt.gz"):
  data.append(e)


In [55]:
#read the data as pandas dataframe
df = pd.DataFrame(data)
#rename columns
df.columns = ['productID','product','price','userID','profile','helpfulness','score','date','summary','text']

#Replace all unknown values with None
df = df.replace({'unknown': None})

#Convert column types
df[['price','score','date']] = df[['price','score','date']].apply(pd.to_numeric)
df['date'] = pd.to_datetime(df['date'], unit='s')
df[['productID','product','userID','profile','helpfulness','summary','text']] = \
    df[['productID','product','userID','profile','helpfulness','summary','text']].astype("string")


In [58]:
df.head()


,productID,product,price,userID,profile,helpfulness,score,date,summary,text
0,B000JVER7W,Mobile Action MA730 Handset Manager - Bluetoot...,NaN,A1RXYH9ROBAKEZ,A. Igoe,0/0,1.0,2009-01-31,Don't buy!,"First of all, the company took my money and se..."
1,B000JVER7W,Mobile Action MA730 Handset Manager - Bluetoot...,NaN,A7L6E1KSJTAJ6,Steven Martz,0/0,5.0,2007-10-04,Mobile Action Bluetooth Mobile Phone Tool Soft...,Great product- tried others and this is a ten ...
2,B000JVER7W,Mobile Action MA730 Handset Manager - Bluetoot...,NaN,A2NOQQ7I6PU7CW,"Daniel M. Johnson ""rocknbluesharp""",0/0,4.0,2007-08-10,good,works real good....a little hard to set up...w...
3,B000924R5I,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",NaN,A29Q0CLOF0U8BN,E. Owens,4/5,4.0,2006-04-28,No instructions included...,The price was right for this cable ($11.95+$4....
4,B000924R5I,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",NaN,ANNHJND2MHUI3,"Isaac Salas ""=CRBF=gB^link""",0/0,1.0,2007-03-12,NOT A DATA CABLE,this is NOT a DATA CABLE this is only a USB ch...
